In [ ]:
#KEEP this one
from pdf2image import convert_from_path
from PIL import Image, ImageDraw
import pytesseract
import cv2
import numpy as np
import pandas as pd

In [7]:
#Select only the account number
from pdf2image import convert_from_path
from PIL import Image, ImageDraw
import pytesseract
import cv2
import numpy as np
import re
import pandas as pd

# Step 1: Convert all pages of the PDF to images
images = convert_from_path('Signed_TestFraudAffidavit.pdf', dpi=300)

# Initialize a list to store data for the DataFrame
data = []

for page_num, image in enumerate(images, start=1):
    print(f"Processing page {page_num}...")
    
    # Step: Preprocess the entire image for better OCR accuracy
    image_np = np.array(image)  # Convert the full image to a NumPy array
    gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)  # Convert the image to grayscale
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)  # Apply thresholding
    preprocessed_image = Image.fromarray(thresh)  # Convert the thresholded array back to a PIL image
    
    # Perform OCR on the page
    ocr_text = pytesseract.image_to_string(preprocessed_image)
    
    # Check if the target string is in the OCR text and group accordingly
    if "FRAUD INVESTIGATION AFFIDAVIT" in ocr_text:
        # Attempt to find "Account Number" and its associated value
        account_number = None
        match = re.search(r"Account Number\s*[:\-]?\s*(\S+)", ocr_text)
        if match:
            account_number = match.group(1)
        
        # Append the results to the data list
        data.append({
            "page_number": page_num,
            "account_number": account_number
        })
        print(f"Page {page_num} identified as Customer Information type")
        if account_number:
            print(f"  Account Number found: {account_number}")
        else:
            print(f"  Account Number not found.")
#     else:
#         # Add to other pages data if "FRAUD INVESTIGATION AFFIDAVIT" is not found
#         data.append({
#             "page_number": page_num,
#             "account_number": None,
#             "text": ocr_text
#         })
#         print(f"Page {page_num} identified as Other type")

# Convert the data into a DataFrame
df = pd.DataFrame(data)

# Print the DataFrame to see the results
print("\nDataFrame with Account Numbers:")
print(df)

# Optionally, you can save the DataFrame to a CSV file
# df.to_csv("account_numbers.csv", index=False)


Processing page 1...
Page 1 identified as Customer Information type
  Account Number found: 999999999
Processing page 2...
Processing page 3...
Processing page 4...

DataFrame with Account Numbers:
   page_number account_number
0            1      999999999


In [8]:
df.head()

,page_number,account_number
0,1,999999999


In [ ]:
#KEEP this one
from pdf2image import convert_from_path
from PIL import Image, ImageDraw
import pytesseract
import cv2
import numpy as np

# # Helper function to convert points to pixels using DPI
# def points_to_pixels(points, dpi=300):
#     return (points / 72) * dpi

# # Step 1: Convert all pages of the PDF to images
images = convert_from_path('Signed_TestFraudAffidavit.pdf', dpi=300)

# initial_boundaries = {
#     1: {"left": 38, "top": 90, "width": 540, "height": 225},  # Covers all fields for page 1
#     2: {"left": 50, "top": 140, "width": 500, "height": 200}   # Example boundary for page 2
# }

# Dictionary to store results by page type
grouped_results = {
    "page_1": [],  # Will store results from pages containing the target string
    "other_pages": []  # Will store results from other pages
}

for page_num, image in enumerate(images, start=1):
    print(f"Processing page {page_num}...")
    
    # Step: Preprocess the entire image for better OCR accuracy
    image_np = np.array(image)  # Convert the full image to a NumPy array
    gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)  # Convert the image to grayscale
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)  # Apply thresholding
    preprocessed_image = Image.fromarray(thresh)  # Convert the thresholded array back to a PIL image
    
    # Perform OCR on the page
    ocr_text = pytesseract.image_to_string(preprocessed_image)
    
    # Check if the target string is in the OCR text and group accordingly
    if "FRAUD INVESTIGATION AFFIDAVIT" in ocr_text:
        grouped_results["page_1"].append({
            "page_number": page_num,
            "text": ocr_text
        })
        print(f"Page {page_num} identified as Customer Information type")
    else:
        grouped_results["other_pages"].append({
            "page_number": page_num,
            "text": ocr_text
        })
        print(f"Page {page_num} identified as Other type")

# Print summary of results
# print("\nResults Summary:")
# print(f"Found {len(grouped_results['page_1'])} pages containing 'FRAUD INVESTIGATION AFFIDAVIT'")
# print(f"Found {len(grouped_results['other_pages'])} other pages")

# # You can access the results like this:
# print("\nPage 1 type documents:")
# for result in grouped_results["page_1"]:
#     print(f"Page {result['page_number']}:")
#     print("-" * 50)
#     print(result["text"])
#     print("-" * 50)

In [ ]:
def find_text_position(text, data):
    positions = []
    for i, word in enumerate(data['text']):
        if text.lower() in word.lower():  # Case-insensitive match
            # Return the bounding box of the word containing the text
            positions.append({
                'left': data['left'][i],
                'top': data['top'][i],
                'width': data['width'][i],
                'height': data['height'][i]
            })
    return positions

In [ ]:
text_to_find = "Account Holder"
boundary = find_text_position(text_to_find, ocr_text)

if boundary:
    print(f"Found {text_to_find} at: {boundary}")
else:
    print(f"Could not find {text_to_find}")


In [ ]:
# List to store individual rows
individual_results = []

# Process page_1 results
for result in grouped_results["page_1"]:
    # Add page number as a separate row
    individual_results.append(f"Page {result['page_number']}:")
    
    # Add separator as a separate row
    individual_results.append("-" * 50)
    
    # Split the text into individual rows by newline (to break the content into rows)
    text_lines = result['text'].splitlines()
    for line in text_lines:
        individual_results.append(line)
    
    # Add another separator as a separate row after the text
    individual_results.append("-" * 50)

# # Process other_pages results in the same way
# for result in grouped_results["other_pages"]:
#     individual_results.append(f"Page {result['page_number']}:")
#     individual_results.append("-" * 50)
#     text_lines = result['text'].splitlines()
#     for line in text_lines:
#         individual_results.append(line)
#     individual_results.append("-" * 50)

# # Convert the list into a DataFrame
# df = pd.DataFrame(individual_results, columns=["Result"])

# # Print the DataFrame
# print(df)


In [ ]:
individual_results

In [ ]:
# import pandas as pd
# import re

# # Initialize a dictionary to store the extracted information
# extracted_data = {
#     "Customer Name": [],
#     "Account Number": [],
#     "Phone Number": [],
#     "Address": [],
#     "City": [],
#     "Report Number": [],
#     "Employee Name": []
# }

# # Loop through the rows in individual_results and extract relevant data
# for row in individual_results:
#     # Search for customer name (assuming it's the text after "Account Holder(s)")
#     if "Account Holder(s)" in row:
#         customer_name = individual_results[individual_results.index(row) + 1].strip()
#         extracted_data["Customer Name"].append(customer_name)

#     # Search for account number (assuming it's the text after "Account Number")
#     if "Account Number" in row:
#         account_number = individual_results[individual_results.index(row) + 1].strip()
#         extracted_data["Account Number"].append(account_number)

#     # Search for phone number (assuming it's the text after "Phone Number")
#     if "Phone Number" in row:
#         phone_number = individual_results[individual_results.index(row) + 1].strip()
#         extracted_data["Phone Number"].append(phone_number)

#     # Search for address (assuming it's the text after "Address")
#     if "Address" in row:
#         address = individual_results[individual_results.index(row) + 1].strip()
#         extracted_data["Address"].append(address)

#     # Search for city (assuming it's the text after "City")
#     if "City" in row:
#         city = individual_results[individual_results.index(row) + 1].strip()
#         extracted_data["City"].append(city)

#     # Search for report number (assuming it's the text after "Report Number")
#     if "Report Number" in row:
#         report_number = individual_results[individual_results.index(row) + 1].strip()
#         extracted_data["Report Number"].append(report_number)

#     # Search for employee name (assuming it's the text after "Employee Name")
#     if "Employee Name" in row:
#         employee_name = individual_results[individual_results.index(row) + 1].strip()
#         extracted_data["Employee Name"].append(employee_name)

# # Convert the extracted data into a DataFrame
# df = pd.DataFrame(extracted_data)

# # Print the DataFrame
# print(df)


In [ ]:
import pandas as pd

# Initialize a dictionary to store the extracted information
extracted_data = {
    "Customer Name": [],
    "Account Number": [],
    "Phone Number": [],
    "Address": [],
    "City": [],
    "Report Number": [],
    "Employee Name": []
}

# Loop through the rows in individual_results and extract relevant data
for i, row in enumerate(individual_results):
    # Combine current row and next row (if exists) into a single block of text
    combined_row = row + " " + (individual_results[i + 1] if i + 1 < len(individual_results) else "")

    # Customer Name: After "Account Holder(s)"
    if "Account Holder(s)" in row:
        # Extract from the combined row
        customer_name = combined_row.split("Account Holder(s)")[1].strip().split("\n")[0]
        extracted_data["Customer Name"].append(customer_name)

    # Account Number: After "Account Number"
    if "Account Number" in row:
        account_number = combined_row.split("Account Number")[1].strip().split("\n")[0]
        extracted_data["Account Number"].append(account_number)

    # Phone Number: After "Phone Number"
    if "Phone Number" in row:
        phone_number = combined_row.split("Phone Number")[1].strip().split("\n")[0]
        extracted_data["Phone Number"].append(phone_number)

    # Address: After "Address"
    if "Address" in row:
        address = combined_row.split("Address")[1].strip().split("\n")[0]
        extracted_data["Address"].append(address)

    # City: After "City"
    if "City" in row:
        city = combined_row.split("City")[1].strip().split("\n")[0]
        extracted_data["City"].append(city)
        
    #State: After "State"
    if "State" in row:
        state = combine_row.split("State")[1].strip.split("\n")[0]
        extracted_data["State"].append(state)

    # Report Number: After "Report Number"
    if "Report Number" in row:
        report_number = combined_row.split("Report Number")[1].strip().split("\n")[0]
        extracted_data["Report Number"].append(report_number)

    # Employee Name: After "Employee Name"
    if "Employee Name" in row:
        employee_name = combined_row.split("Employee Name")[1].strip().split("\n")[0]
        extracted_data["Employee Name"].append(employee_name)

# Convert the extracted data into a DataFrame
df = pd.DataFrame(extracted_data)

# Print the DataFrame
# print(df)


In [ ]:
df.head()

Radio button search below

In [ ]:
import pytesseract
import cv2
import numpy as np
from pdf2image import convert_from_path

def extract_image_from_pdf(pdf_path, page_number=2):
    # Convert the PDF page to an image (list of PIL images)
    pages = convert_from_path(pdf_path)
    # Convert the first page to OpenCV format (numpy array)
    img = np.array(pages[page_number])
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

def find_radio_button_in_pdf(pdf_path, keyword, page_number=2):
    # Extract image from the PDF
    img = extract_image_from_pdf(pdf_path, page_number)
    
    # Convert the image to grayscale for easier processing
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Perform OCR to extract the text and its bounding boxes
    h, w, _ = img.shape
    boxes = pytesseract.image_to_boxes(img)  # Returns the bounding boxes of the detected text
    
    # Search for the keyword's position
    keyword_position = None
    for b in boxes.splitlines():
        b = b.split()
        if b[0] == keyword:
            x1, y1, x2, y2 = int(b[1]), int(b[2]), int(b[3]), int(b[4])
            keyword_position = (x1, y1, x2, y2)
            break
    
    if not keyword_position:
        print(f"Keyword '{keyword}' not found.")
        return None
    
    # Define a region to look left of the keyword position for the radio button
    x1, y1, x2, y2 = keyword_position
    region_left = img[y1-20:y2+20, x1-80:x1]  # Look left of the keyword with some padding
    
    # Convert the region to grayscale
    region_gray = cv2.cvtColor(region_left, cv2.COLOR_BGR2GRAY)
    
    # Thresholding to detect dark (shaded) regions (you may need to adjust the threshold)
    _, thresh = cv2.threshold(region_gray, 50, 255, cv2.THRESH_BINARY_INV)
    
    # Find contours in the thresholded image (for detecting circular shapes)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Loop through contours and check if they match a radio button (circular shape)
    for contour in contours:
        # Fit a circle to the contour
        (x, y), radius = cv2.minEnclosingCircle(contour)
        
        # Check if the contour is approximately circular and dark
        if 10 < radius < 30:  # Radius of radio button; adjust as necessary
            if np.mean(region_gray[int(y-radius):int(y+radius), int(x-radius):int(x+radius)]) < 100:  # Check shading
                print(f"Radio button detected at {(x1 - 80 + x, y1 + y)}")
                return (x1 - 80 + x, y1 + y)  # Return the position of the radio button
    
    print("No radio button detected.")
    return None

# Example usage
pdf_path = 'Signed_TestFraudAffidavit.pdf'  # Replace with your PDF path
keyword = 'Forged Signature'  # Replace with your keyword
find_radio_button_in_pdf(pdf_path, keyword)
